In [1]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
from PIL import Image
from glob import glob

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', required=True, help='cifar10 | lsun | imagenet | folder | lfw ')
parser.add_argument('--dataroot', required=True, help='path to dataset')
parser.add_argument('--workers', type=int, help='number of data loading workers', default=2)
parser.add_argument('--batch_size', type=int, default=16, help='input batch size')
parser.add_argument('--image_size', type=int, default=64, help='the height / width of the input image to network')
parser.add_argument('--nz', type=int, default=64, help='size of the latent z vector')
parser.add_argument('--niter', type=int, default=25, help='number of epochs to train for')
parser.add_argument('--lr', type=float, default=0.0001, help='learning rate, default=0.0002')
parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for adam. default=0.5')
parser.add_argument('--cuda', action='store_true', help='enables cuda')
parser.add_argument('--netD', default='', help="path to netD (to continue training)")
parser.add_argument('--netG', default='', help="path to netG (to continue training)")
parser.add_argument('--base_dir', default='./train4/', help='directory to save image/model/data')
parser.add_argument('--k_init', type=float, default=0.0, help='initial value of k')
parser.add_argument('--gamma', type=float, default=0.4, help='diversity ratio')
parser.add_argument('--numOfKernel', type=int, default=64, help='Number of Kernels in G\'s Conv')
parser.add_argument('--lr_decay', type=float, default=0.95, help='learning rate decay rate')

_StoreAction(option_strings=['--lr_decay'], dest='lr_decay', nargs=None, const=None, default=0.95, type=<type 'float'>, choices=None, help='learning rate decay rate', metavar=None)

In [3]:
opt = parser.parse_args(['--dataset', 'folder', '--dataroot', '/home/ubuntu/data/work/', '--cuda', '--niter', '25'])
print(opt)

Namespace(base_dir='./train4/', batch_size=16, beta1=0.5, cuda=True, dataroot='/home/ubuntu/data/work/', dataset='folder', gamma=0.4, image_size=64, k_init=0.0, lr=0.0001, lr_decay=0.95, netD='', netG='', niter=25, numOfKernel=64, nz=64, workers=2)


In [4]:
try:
    os.makedirs(opt.base_dir)
except OSError:
    pass

In [5]:
# if opt.manualSeed is None:
#     opt.manualSeed = random.randint(1, 10000)
# print("Random Seed: ", opt.manualSeed)
# random.seed(opt.manualSeed)
# torch.manual_seed(opt.manualSeed)
# if opt.cuda:
#     torch.cuda.manual_seed_all(opt.manualSeed)

In [6]:
cudnn.benchmark = True
if torch.cuda.is_available() and not opt.cuda:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")

In [7]:
torch.cuda.set_device(0)

In [8]:
file_list = glob('/home/ubuntu/data/work/img_align_celeba/*')
mini_batch = len(file_list) / opt.batch_size
def get_permutation():
    return np.random.permutation(len(file_list))
def get_samples(index, i):
    data = torch.FloatTensor(opt.batch_size, 3, opt.image_size, opt.image_size)
    j = 0
    for i in range(opt.batch_size*i, opt.batch_size*(i+1)):
        data[j] = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(transforms.ToTensor()(Image.open(file_list[index[i]])))
        j += 1
    return data

In [9]:
ngpu = 1
nz = int(opt.nz)
k = opt.k_init

In [10]:
# custom weights initialization called on netG and netD
def G_weights_init(m):
    classname = m.__class__.__name__

    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.002)

    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

def D_weights_init(m):
    classname = m.__class__.__name__

    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)

    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [11]:
class G_CNN(nn.Module):
    def __init__(self, embedding_size, kernel_num):
        super(G_CNN, self).__init__()
        self.kernel_num = kernel_num
        FC_start_size = 8 * 8 * self.kernel_num
        self.start = nn.Linear(embedding_size, FC_start_size)
        #all layers
        self.main_arch = nn.Sequential(
            #repeated units
            nn.Conv2d(in_channels=kernel_num, out_channels=kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=kernel_num, out_channels=kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.UpsamplingNearest2d(scale_factor=2),
            #repeated units
            nn.Conv2d(in_channels=kernel_num, out_channels=kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=kernel_num, out_channels=kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.UpsamplingNearest2d(scale_factor=2),
            #repeated units
            nn.Conv2d(in_channels=kernel_num, out_channels=kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=kernel_num, out_channels=kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.UpsamplingNearest2d(scale_factor=2),
            #last layer
            #nn.Conv2d(in_channels=kernel_num, out_channels=3, kernel_size=3, padding=1)
            nn.Conv2d(in_channels=kernel_num, out_channels=kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=kernel_num, out_channels=kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=kernel_num, out_channels=3, kernel_size=3, padding=1)
        )

    def forward(self, in_data):
        in_data = self.start(in_data.view(-1, 64))
        in_data = in_data.view(-1, self.kernel_num, 8, 8)
        output = self.main_arch(in_data)
        return output

In [12]:
netG = G_CNN(embedding_size=nz, kernel_num=opt.numOfKernel)
netG.apply(G_weights_init)
if opt.netG != '':
    netG.load_state_dict(torch.load(opt.netG))

In [13]:
#Discriminator
class D_CNN(nn.Module):
    def __init__(self, embedding_size, base_kernel_num):
        super(D_CNN, self).__init__()
        self.base_kernel_num = base_kernel_num
        
        #encoder w/o flatten and FC
        self.encoder_wo_FC = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            
            nn.Conv2d(in_channels=base_kernel_num, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=base_kernel_num, out_channels=2*base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            #downsampling
            nn.Conv2d(in_channels=2*base_kernel_num, out_channels=2*base_kernel_num, kernel_size=3, padding=1, stride=2),
            nn.ELU(inplace=True),
            
            nn.Conv2d(in_channels=2*base_kernel_num, out_channels=2*base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=2*base_kernel_num, out_channels=3*base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            #downsampling
            nn.Conv2d(in_channels=3*base_kernel_num, out_channels=3*base_kernel_num, kernel_size=3, padding=1, stride=2),
            nn.ELU(inplace=True),
            
            nn.Conv2d(in_channels=3*base_kernel_num, out_channels=3*base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=3*base_kernel_num, out_channels=4*base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            #downsampling
            nn.Conv2d(in_channels=4*base_kernel_num, out_channels=4*base_kernel_num, kernel_size=3, padding=1, stride=2),
            nn.ELU(inplace=True),
            
            nn.Conv2d(in_channels=4*base_kernel_num, out_channels=4*base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=4*base_kernel_num, out_channels=4*base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True)
        )
        FC_mid1_size = 8 * 8 * 4 * self.base_kernel_num
        FC_mid2_size = 8 * 8 * self.base_kernel_num
        self.FC1 = nn.Linear(FC_mid1_size, embedding_size)
        self.FC2 = nn.Linear(embedding_size, FC_mid2_size)
        self.decoder_wo_FC = nn.Sequential(
            #repeated units
            nn.Conv2d(in_channels=base_kernel_num, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=base_kernel_num, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.UpsamplingNearest2d(scale_factor=2),
            #repeated units
            nn.Conv2d(in_channels=base_kernel_num, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=base_kernel_num, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.UpsamplingNearest2d(scale_factor=2),
            #repeated units
            nn.Conv2d(in_channels=base_kernel_num, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=base_kernel_num, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.UpsamplingNearest2d(scale_factor=2),
            #last layer
            #nn.Conv2d(in_channels=base_kernel_num, out_channels=3, kernel_size=3, padding=1)
            nn.Conv2d(in_channels=base_kernel_num, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=base_kernel_num, out_channels=base_kernel_num, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=base_kernel_num, out_channels=3, kernel_size=3, padding=1)
        )

    def forward(self, in_data):
        in_data = self.encoder_wo_FC(in_data)
        
        in_data = in_data.view(-1, 4*self.base_kernel_num * 8 * 8);
        in_data = self.FC1(in_data)
        in_data = self.FC2(in_data)
        in_data = in_data.view(-1, self.base_kernel_num, 8, 8)
        
        output = self.decoder_wo_FC(in_data)
        return output

In [14]:
netD = D_CNN(embedding_size=nz, base_kernel_num=opt.numOfKernel)
netD.apply(D_weights_init)
if opt.netD != '':
    netD.load_state_dict(torch.load(opt.netD))

In [15]:
criterion_L1 = nn.L1Loss()

input = torch.FloatTensor(opt.batch_size, 3, opt.image_size, opt.image_size)
noise = torch.FloatTensor(opt.batch_size, nz)
fixed_noise = torch.FloatTensor(opt.batch_size, nz, 1, 1).uniform_(-1, 1)

if opt.cuda:
    netD.cuda()
    netG.cuda()
    criterion_L1.cuda()
    input = input.cuda()
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

input = Variable(input)
noise = Variable(noise)
fixed_noise = Variable(fixed_noise)
np.save(opt.base_dir+'fixed_noise.npy', fixed_noise.data.cpu().numpy())
# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=opt.lr, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=opt.lr, betas=(0.5, 0.999))

In [ ]:
opt.lr

0.0001

In [ ]:
data_loss_D = []
data_loss_G = []
data_loss_k = []
data_loss_global = []
for epoch in range(opt.niter):
    index = get_permutation()
    for i in range(int(mini_batch)):

        ############################
        # (1) Update D network
        ###########################
        netD.zero_grad()

        # prepare real
        input.data.copy_(get_samples(index, i))
#         real_cpu, _ = data
#         batch_size = real_cpu.size(0)
#         input.data.resize_(real_cpu.size()).copy_(real_cpu)

        # train with real
        output = netD(input)
        errD_real = criterion_L1(output, input)  # score on real
        errD_real.backward()  # backward on score on real
        L_x = errD_real.data[0]  # score fore supervision
        
        # generate fake
        #noise.data.resize_(batch_size, nz, 1, 1)
        noise.data.uniform_(-1, 1)
        fake = netG(noise)
        
        # train with fake
        output = netD(fake.detach())
        errD_fake = criterion_L1(output, fake.detach())  # score on fake
        errD_fake_use = - k * errD_fake
        errD_fake_use.backward()  # backward on score on fake
        optimizerD.step()
        
        #D_G_z1 = errD_fake.data.mean()  # score fore supervision <- generated when calc D loss
        errD = errD_real + errD_fake_use  # score fore supervision

        

        ############################
        # (2) Update G network
        ############################

        netG.zero_grad()

        # generate fake
#         noise.data.resize_(batch_size, nz, 1, 1)
#         noise.data.uniform_(-1, 1)
        fake = netG(noise)
        
        # NOT reuse generated fake samples
        output = netD(fake)
        errG = (output - fake).abs().mean()  # L1
        errG.backward()
        
        L_G = errG.data[0] # score fore supervision <- generated when calc G loss

        optimizerG.step()
        
        # K STEP
        #left -> real
        output = netD(input)
        errD_real_k = criterion_L1(output, input)  # score on real
        #right -> fake
        fake = netG(noise)
        output = netD(fake.detach())
        errD_fake_k = criterion_L1(output, fake.detach())  # score on fake
        
        #Convergence Measure
        cm = errD_real_k.data[0] + abs(opt.gamma * errD_real_k.data[0] - L_G)
        
        k += 0.001 * (opt.gamma * errD_real_k.data[0] - errD_fake_k.data[0])
        k = max(min(k, 1), 0)
        ############################
        # (3) Report & 100 Batch checkpoint
        ############################
        data_loss_G.append(errG.data[0])
        data_loss_D.append(errD.data[0])
        data_loss_k.append(k)
        data_loss_global.append(cm)
        if i % 2000 == 1:
            optimizerD = optim.Adam(netD.parameters(), lr=opt.lr*opt.lr_decay, betas=(0.5, 0.999))
            optimizerG = optim.Adam(netG.parameters(), lr=opt.lr*opt.lr_decay, betas=(0.5, 0.999))
            opt.lr = opt.lr * opt.lr_decay
        if i % 10 == 0:
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f Convergence: %.4f k= %.6f lr = %.7f'
                    % (epoch, opt.niter, i, int(mini_batch),
                     errD.data[0], errG.data[0], cm, k, opt.lr))
        
        if i % 100 == 0:
            fake = netG(fixed_noise)
            vutils.save_image(fake.data,
                              opt.base_dir+'%d_fake_samples_epoch_%03d.png' % (i, epoch), normalize = True)

    # do checkpointing
    torch.save(netG.state_dict(), opt.base_dir+'netG_epoch_%d.pth' % (epoch))
    torch.save(netD.state_dict(), opt.base_dir+'netD_epoch_%d.pth' % (epoch))
    np.save(opt.base_dir+'k.npy', np.array(k))

[0/25][0/9765] Loss_D: 0.4689 Loss_G: 0.0114 Convergence: 0.6427 k= 0.000176 lr = 0.0001000
[0/25][10/9765] Loss_D: 0.4100 Loss_G: 0.1382 Convergence: 0.4176 k= 0.001508 lr = 0.0000950
[0/25][20/9765] Loss_D: 0.3795 Loss_G: 0.2455 Convergence: 0.4699 k= 0.002028 lr = 0.0000950
[0/25][30/9765] Loss_D: 0.3667 Loss_G: 0.1746 Convergence: 0.3875 k= 0.001169 lr = 0.0000950
[0/25][40/9765] Loss_D: 0.3381 Loss_G: 0.2744 Convergence: 0.4683 k= 0.000000 lr = 0.0000950
[0/25][50/9765] Loss_D: 0.4019 Loss_G: 0.1036 Convergence: 0.4464 k= 0.000446 lr = 0.0000950
[0/25][60/9765] Loss_D: 0.2950 Loss_G: 0.0845 Convergence: 0.3132 k= 0.000592 lr = 0.0000950
[0/25][70/9765] Loss_D: 0.2792 Loss_G: 0.0988 Convergence: 0.2864 k= 0.000784 lr = 0.0000950
[0/25][80/9765] Loss_D: 0.2839 Loss_G: 0.0796 Convergence: 0.3078 k= 0.001002 lr = 0.0000950
[0/25][90/9765] Loss_D: 0.2670 Loss_G: 0.0970 Convergence: 0.2593 k= 0.001218 lr = 0.0000950
[0/25][100/9765] Loss_D: 0.2889 Loss_G: 0.0928 Convergence: 0.2929 k= 0

[0/25][880/9765] Loss_D: 0.1662 Loss_G: 0.0822 Convergence: 0.1816 k= 0.000004 lr = 0.0000950
[0/25][890/9765] Loss_D: 0.1843 Loss_G: 0.1251 Convergence: 0.2302 k= 0.000000 lr = 0.0000950
[0/25][900/9765] Loss_D: 0.1905 Loss_G: 0.1057 Convergence: 0.2138 k= 0.000000 lr = 0.0000950
[0/25][910/9765] Loss_D: 0.1874 Loss_G: 0.0683 Convergence: 0.1804 k= 0.000017 lr = 0.0000950
[0/25][920/9765] Loss_D: 0.1834 Loss_G: 0.2145 Convergence: 0.3413 k= 0.000000 lr = 0.0000950
[0/25][930/9765] Loss_D: 0.1985 Loss_G: 0.0451 Convergence: 0.2237 k= 0.000075 lr = 0.0000950
[0/25][940/9765] Loss_D: 0.1593 Loss_G: 0.0687 Convergence: 0.1651 k= 0.000218 lr = 0.0000950
[0/25][950/9765] Loss_D: 0.1875 Loss_G: 0.0736 Convergence: 0.1848 k= 0.000215 lr = 0.0000950
[0/25][960/9765] Loss_D: 0.1656 Loss_G: 0.0378 Convergence: 0.1886 k= 0.000369 lr = 0.0000950
[0/25][970/9765] Loss_D: 0.1605 Loss_G: 0.0313 Convergence: 0.1954 k= 0.000673 lr = 0.0000950
[0/25][980/9765] Loss_D: 0.1560 Loss_G: 0.0330 Convergence: 

[0/25][1750/9765] Loss_D: 0.1594 Loss_G: 0.0584 Convergence: 0.1742 k= 0.001700 lr = 0.0000950
[0/25][1760/9765] Loss_D: 0.1489 Loss_G: 0.0597 Convergence: 0.1482 k= 0.001645 lr = 0.0000950
[0/25][1770/9765] Loss_D: 0.1595 Loss_G: 0.0623 Convergence: 0.1579 k= 0.001602 lr = 0.0000950
[0/25][1780/9765] Loss_D: 0.1796 Loss_G: 0.0624 Convergence: 0.1722 k= 0.001670 lr = 0.0000950
[0/25][1790/9765] Loss_D: 0.1594 Loss_G: 0.0566 Convergence: 0.1639 k= 0.001599 lr = 0.0000950
[0/25][1800/9765] Loss_D: 0.1749 Loss_G: 0.0647 Convergence: 0.1723 k= 0.001566 lr = 0.0000950
[0/25][1810/9765] Loss_D: 0.1666 Loss_G: 0.0544 Convergence: 0.1665 k= 0.001548 lr = 0.0000950
[0/25][1820/9765] Loss_D: 0.1648 Loss_G: 0.0699 Convergence: 0.1669 k= 0.001488 lr = 0.0000950
[0/25][1830/9765] Loss_D: 0.1566 Loss_G: 0.0588 Convergence: 0.1626 k= 0.001387 lr = 0.0000950
[0/25][1840/9765] Loss_D: 0.1637 Loss_G: 0.0732 Convergence: 0.1663 k= 0.001360 lr = 0.0000950
[0/25][1850/9765] Loss_D: 0.1532 Loss_G: 0.0681 Co

[0/25][2620/9765] Loss_D: 0.1443 Loss_G: 0.0553 Convergence: 0.1435 k= 0.004853 lr = 0.0000902
[0/25][2630/9765] Loss_D: 0.1574 Loss_G: 0.0541 Convergence: 0.1580 k= 0.004874 lr = 0.0000902
[0/25][2640/9765] Loss_D: 0.1448 Loss_G: 0.0504 Convergence: 0.1434 k= 0.004845 lr = 0.0000902
[0/25][2650/9765] Loss_D: 0.1308 Loss_G: 0.0529 Convergence: 0.1297 k= 0.004933 lr = 0.0000902
[0/25][2660/9765] Loss_D: 0.1224 Loss_G: 0.0592 Convergence: 0.1307 k= 0.004884 lr = 0.0000902
[0/25][2670/9765] Loss_D: 0.1433 Loss_G: 0.0590 Convergence: 0.1428 k= 0.004933 lr = 0.0000902
[0/25][2680/9765] Loss_D: 0.1517 Loss_G: 0.0519 Convergence: 0.1506 k= 0.004947 lr = 0.0000902
[0/25][2690/9765] Loss_D: 0.1452 Loss_G: 0.0534 Convergence: 0.1463 k= 0.004935 lr = 0.0000902
[0/25][2700/9765] Loss_D: 0.1315 Loss_G: 0.0554 Convergence: 0.1334 k= 0.004956 lr = 0.0000902
[0/25][2710/9765] Loss_D: 0.1392 Loss_G: 0.0621 Convergence: 0.1511 k= 0.004956 lr = 0.0000902
[0/25][2720/9765] Loss_D: 0.1366 Loss_G: 0.0492 Co

[0/25][3490/9765] Loss_D: 0.1278 Loss_G: 0.0465 Convergence: 0.1331 k= 0.008734 lr = 0.0000902
[0/25][3500/9765] Loss_D: 0.1341 Loss_G: 0.0456 Convergence: 0.1395 k= 0.008768 lr = 0.0000902
[0/25][3510/9765] Loss_D: 0.1304 Loss_G: 0.0452 Convergence: 0.1351 k= 0.008790 lr = 0.0000902
[0/25][3520/9765] Loss_D: 0.1266 Loss_G: 0.0445 Convergence: 0.1373 k= 0.008816 lr = 0.0000902
[0/25][3530/9765] Loss_D: 0.1338 Loss_G: 0.0523 Convergence: 0.1307 k= 0.008848 lr = 0.0000902
[0/25][3540/9765] Loss_D: 0.1387 Loss_G: 0.0631 Convergence: 0.1446 k= 0.008920 lr = 0.0000902
[0/25][3550/9765] Loss_D: 0.1291 Loss_G: 0.0430 Convergence: 0.1335 k= 0.009004 lr = 0.0000902
[0/25][3560/9765] Loss_D: 0.1513 Loss_G: 0.0681 Convergence: 0.1623 k= 0.009069 lr = 0.0000902
[0/25][3570/9765] Loss_D: 0.1374 Loss_G: 0.0536 Convergence: 0.1436 k= 0.009101 lr = 0.0000902
[0/25][3580/9765] Loss_D: 0.1343 Loss_G: 0.0502 Convergence: 0.1334 k= 0.009078 lr = 0.0000902
[0/25][3590/9765] Loss_D: 0.1495 Loss_G: 0.0456 Co

...

[0/25][8710/9765] Loss_D: 0.1263 Loss_G: 0.0461 Convergence: 0.1254 k= 0.030277 lr = 0.0000774
[0/25][8720/9765] Loss_D: 0.1198 Loss_G: 0.0566 Convergence: 0.1300 k= 0.030259 lr = 0.0000774
[0/25][8730/9765] Loss_D: 0.1177 Loss_G: 0.0387 Convergence: 0.1245 k= 0.030306 lr = 0.0000774
[0/25][8740/9765] Loss_D: 0.1120 Loss_G: 0.0450 Convergence: 0.1111 k= 0.030354 lr = 0.0000774
[0/25][8750/9765] Loss_D: 0.1086 Loss_G: 0.0536 Convergence: 0.1182 k= 0.030331 lr = 0.0000774
[0/25][8760/9765] Loss_D: 0.1172 Loss_G: 0.0413 Convergence: 0.1222 k= 0.030371 lr = 0.0000774
[0/25][8770/9765] Loss_D: 0.1207 Loss_G: 0.0516 Convergence: 0.1241 k= 0.030404 lr = 0.0000774
[0/25][8780/9765] Loss_D: 0.1276 Loss_G: 0.0437 Convergence: 0.1275 k= 0.030430 lr = 0.0000774
[0/25][8790/9765] Loss_D: 0.1181 Loss_G: 0.0434 Convergence: 0.1201 k= 0.030453 lr = 0.0000774
[0/25][8800/9765] Loss_D: 0.1175 Loss_G: 0.0452 Convergence: 0.1181 k= 0.030466 lr = 0.0000774
[0/25][8810/9765] Loss_D: 0.1183 Loss_G: 0.0545 Co

[0/25][9580/9765] Loss_D: 0.1227 Loss_G: 0.0597 Convergence: 0.1366 k= 0.032114 lr = 0.0000774
[0/25][9590/9765] Loss_D: 0.1275 Loss_G: 0.0521 Convergence: 0.1287 k= 0.032149 lr = 0.0000774
[0/25][9600/9765] Loss_D: 0.1191 Loss_G: 0.0444 Convergence: 0.1228 k= 0.032148 lr = 0.0000774
[0/25][9610/9765] Loss_D: 0.1097 Loss_G: 0.0400 Convergence: 0.1116 k= 0.032156 lr = 0.0000774
[0/25][9620/9765] Loss_D: 0.1238 Loss_G: 0.0440 Convergence: 0.1299 k= 0.032194 lr = 0.0000774
[0/25][9630/9765] Loss_D: 0.1149 Loss_G: 0.0406 Convergence: 0.1201 k= 0.032242 lr = 0.0000774
[0/25][9640/9765] Loss_D: 0.1270 Loss_G: 0.0482 Convergence: 0.1246 k= 0.032286 lr = 0.0000774
[0/25][9650/9765] Loss_D: 0.1243 Loss_G: 0.0566 Convergence: 0.1294 k= 0.032295 lr = 0.0000774
[0/25][9660/9765] Loss_D: 0.1155 Loss_G: 0.0412 Convergence: 0.1183 k= 0.032308 lr = 0.0000774
[0/25][9670/9765] Loss_D: 0.1171 Loss_G: 0.0470 Convergence: 0.1165 k= 0.032343 lr = 0.0000774
[0/25][9680/9765] Loss_D: 0.1116 Loss_G: 0.0441 Co

[1/25][690/9765] Loss_D: 0.1178 Loss_G: 0.0532 Convergence: 0.1232 k= 0.032797 lr = 0.0000735
[1/25][700/9765] Loss_D: 0.1222 Loss_G: 0.0484 Convergence: 0.1227 k= 0.032810 lr = 0.0000735
[1/25][710/9765] Loss_D: 0.1182 Loss_G: 0.0616 Convergence: 0.1348 k= 0.032831 lr = 0.0000735
[1/25][720/9765] Loss_D: 0.1135 Loss_G: 0.0456 Convergence: 0.1129 k= 0.032837 lr = 0.0000735
[1/25][730/9765] Loss_D: 0.1177 Loss_G: 0.0426 Convergence: 0.1201 k= 0.032838 lr = 0.0000735
[1/25][740/9765] Loss_D: 0.1184 Loss_G: 0.0460 Convergence: 0.1180 k= 0.032834 lr = 0.0000735
[1/25][750/9765] Loss_D: 0.1059 Loss_G: 0.0520 Convergence: 0.1148 k= 0.032834 lr = 0.0000735
[1/25][760/9765] Loss_D: 0.1130 Loss_G: 0.0545 Convergence: 0.1229 k= 0.032846 lr = 0.0000735
[1/25][770/9765] Loss_D: 0.1322 Loss_G: 0.0556 Convergence: 0.1335 k= 0.032839 lr = 0.0000735
[1/25][780/9765] Loss_D: 0.1154 Loss_G: 0.0493 Convergence: 0.1183 k= 0.032803 lr = 0.0000735
[1/25][790/9765] Loss_D: 0.1230 Loss_G: 0.0521 Convergence: 

[1/25][1560/9765] Loss_D: 0.1176 Loss_G: 0.0461 Convergence: 0.1170 k= 0.031431 lr = 0.0000735
[1/25][1570/9765] Loss_D: 0.1074 Loss_G: 0.0446 Convergence: 0.1083 k= 0.031410 lr = 0.0000735
[1/25][1580/9765] Loss_D: 0.1178 Loss_G: 0.0406 Convergence: 0.1218 k= 0.031401 lr = 0.0000735
[1/25][1590/9765] Loss_D: 0.1083 Loss_G: 0.0569 Convergence: 0.1247 k= 0.031384 lr = 0.0000735
[1/25][1600/9765] Loss_D: 0.1244 Loss_G: 0.0581 Convergence: 0.1342 k= 0.031386 lr = 0.0000735
[1/25][1610/9765] Loss_D: 0.1182 Loss_G: 0.0494 Convergence: 0.1194 k= 0.031371 lr = 0.0000735
[1/25][1620/9765] Loss_D: 0.1144 Loss_G: 0.0580 Convergence: 0.1270 k= 0.031350 lr = 0.0000735
[1/25][1630/9765] Loss_D: 0.1213 Loss_G: 0.0511 Convergence: 0.1241 k= 0.031316 lr = 0.0000735
[1/25][1640/9765] Loss_D: 0.1223 Loss_G: 0.0502 Convergence: 0.1227 k= 0.031294 lr = 0.0000735
[1/25][1650/9765] Loss_D: 0.1271 Loss_G: 0.0451 Convergence: 0.1318 k= 0.031271 lr = 0.0000735
[1/25][1660/9765] Loss_D: 0.1122 Loss_G: 0.0491 Co

[1/25][2430/9765] Loss_D: 0.1105 Loss_G: 0.0481 Convergence: 0.1144 k= 0.028692 lr = 0.0000698
[1/25][2440/9765] Loss_D: 0.1167 Loss_G: 0.0551 Convergence: 0.1246 k= 0.028651 lr = 0.0000698
[1/25][2450/9765] Loss_D: 0.1180 Loss_G: 0.0460 Convergence: 0.1183 k= 0.028648 lr = 0.0000698
[1/25][2460/9765] Loss_D: 0.1126 Loss_G: 0.0489 Convergence: 0.1154 k= 0.028623 lr = 0.0000698
[1/25][2470/9765] Loss_D: 0.1194 Loss_G: 0.0544 Convergence: 0.1252 k= 0.028587 lr = 0.0000698
[1/25][2480/9765] Loss_D: 0.1134 Loss_G: 0.0463 Convergence: 0.1136 k= 0.028553 lr = 0.0000698
[1/25][2490/9765] Loss_D: 0.1155 Loss_G: 0.0546 Convergence: 0.1236 k= 0.028535 lr = 0.0000698
[1/25][2500/9765] Loss_D: 0.1155 Loss_G: 0.0518 Convergence: 0.1203 k= 0.028503 lr = 0.0000698
[1/25][2510/9765] Loss_D: 0.1202 Loss_G: 0.0501 Convergence: 0.1214 k= 0.028478 lr = 0.0000698
[1/25][2520/9765] Loss_D: 0.1090 Loss_G: 0.0513 Convergence: 0.1168 k= 0.028421 lr = 0.0000698
[1/25][2530/9765] Loss_D: 0.1186 Loss_G: 0.0453 Co